In [4]:
import re
import csv
from collections import Counter, defaultdict

LOG_FILE = "E:\sample.log"
OUTPUT_CSV_FILE = "log_analysis_results.csv"
LOGIN_THRESHOLD = 2

def func_log_file(log_file):
    ip_requests = Counter()
    endpoint_requests = Counter()
    failed_logins = defaultdict(int)
    
    with open(log_file, "r") as file:
        for line in file:
            ip_match = re.search(r"^\d+\.\d+\.\d+\.\d+", line)
            if ip_match:
                ip_address = ip_match.group(0)
                ip_requests[ip_address] += 1
                
            endpoint_match = re.search(r'"(GET|POST|PUT|DELETE|PATCH) (.*?) HTTP/', line)
            if endpoint_match:
                endpoint = endpoint_match.group(2)
                endpoint_requests[endpoint] += 1
                
            if "401" in line or "Invalid credentials" in line:
                if ip_match:
                    failed_logins[ip_address] += 1
                    
    return ip_requests, endpoint_requests, failed_logins

def func_save_file(ip_requests, most_accessed, failed_logins, output_file):
    with open(output_file, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        
        writer.writerow(["IP Address", "Request Count"])
        for ip, count in ip_requests.items():
            writer.writerow([ip, count])
            
        writer.writerow([])
        writer.writerow(['Endpoint', 'Access Count'])
        writer.writerow([most_accessed[0], most_accessed[1]])
        
        writer.writerow([])
        writer.writerow(["IP Address", "Failed Login Count"])
        for ip, count in failed_logins.items():
            if count > LOGIN_THRESHOLD:
                writer.writerow([ip,count])
                
def main():
    ip_requests, endpoint_requests, failed_logins = func_log_file(LOG_FILE)
    
    most_accessed = endpoint_requests.most_common(1)[0]
    
    flagged_ips = {ip: count for ip, count in failed_logins.items() if count > LOGIN_THRESHOLD}
    
    print("IP Address         Request Count")
    for ip, count in ip_requests.most_common():
        print(f"{ip:20} {count}")
        
    print("\nMost Frequently Accessed Endpoint:")
    print(f"{most_accessed[0]} (Accessed {most_accessed[1]} times)")
    
    print("\nSuspicious Activity Detected:")
    print("IP Address          Failed Login Attempts")
    for ip, count in flagged_ips.items():
        print(f"{ip:20} {count}")
        
    func_save_file(ip_requests, most_accessed, flagged_ips, OUTPUT_CSV_FILE)
    print(f"\nResults saved to {OUTPUT_CSV_FILE}")
    
if __name__ == "__main__":
    main()

IP Address         Request Count
203.0.113.5          8
198.51.100.23        8
192.168.1.1          7
10.0.0.2             6
192.168.1.100        5

Most Frequently Accessed Endpoint:
/login (Accessed 13 times)

Suspicious Activity Detected:
IP Address          Failed Login Attempts
203.0.113.5          8
192.168.1.100        5

Results saved to log_analysis_results.csv
